## Biogas plant

You want to plan the two-year supply of raw materials for a biogas power plant. Such a plant produces energy by burning biogas, which is obtained from the bacterial fermentation of organic wastes. 
Specifically, your plant is powered by corn chopping, a residual of agro-industrial operations that you can purchase from 5 local farms. 
The table below shows the quarterly capacity of each farm for the next two years. Quantities are measured in tons.

Farm|T1|T2|T3|T4|T5|T6|T7|T8
:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
1|700|1500|700|0|0|700|1500|0
2|1350|0|450|0|1350|0|450|0
3|0|1500|1500|0|0|1500|1500|0
4|820|1560|820|0|820|1560|820|0
5|0|680|1080|0|0|680|1080|0

Due to crop rotations and corn harvesting periods, farms are unable to supply material in some quarters. Moreover the types of corn chopping provided are different, each coming with its own unitary purchase price, unitary storage cost and percentage of dry matter. The table below shows a summary of these information.

Farm|Purchase price|Storage cost|Dry matter
:-|:-:|:-:|:-:
1|0.20|0.002|15
2|0.18|0.012|28
3|0.19|0.007|35
4|0.21|0.011|37
5|0.23|0.015|42

Your biogas plant must operate by burning a mixture of corn choppings with a dry matter percentage between 20% and 40%. Under these conditions, the yield is 421.6 kWh of energy per ton of burned material. The energy produced by the plant is sold on the market at a price of 0.28 $/kWh. 

Due to state regulations, all biogas plants can produce a maximum of 1950 MWh of energy per quarter. You are allowed to store corn chopping in a silo, whose total capacity is of 500 tons. 

Plan the supply and inventory of your biogas plant with the goal of maximizing your profits (i.e., revenues minus costs).

In [1]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446 kB 7.2 MB/s 
  Attempting uninstall: cffi
    Found existing installation: cffi 1.15.1
    Uninstalling cffi-1.15.1:
      Successfully uninstalled cffi-1.15.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 8.3 MB/s 


In [ ]:
import mip
m = mip.Model()

#SETS
T =   [i for i in range(8)]
T0 =  [i for i in range(-1, 8)]
F =   [i for i in range(5)]

#PARAMETERS
d = [ [   700,    1500,   700,    0,      0,    700,    1500,   0],
      [   1350,   0,      450,    0,      1350, 0,      450,    0],
      [   0,      1500,   1500,   0,      0,    1500,   1500,   0],
      [   820,    1560,   820,    0,      820,  1560,   820,    0],
      [   0,      680,    1080,   0,      0,    680,    1080,   0]]

c =   [   0.2,    0.18,   0.19,   0.21,   0.23  ]
s =   [   0.002,  0.012,  0.007,  0.011,  0.015 ]
dm =  [   15,     28,     35,     37,     42    ]

dmpd = 20
dmpu = 40
kWhpt = 421.6
ppkWh = 0.28
maxeprod = 1950000
maxsc = 500

#VARIABLES
#quantity bought from farm i in quarter t
x = {(i,t) : m.add_var(var_type=mip.CONTINUOUS) for i in F for t in T}
#quantity burned from farm i in quarter t
y = {(i,t) : m.add_var(var_type=mip.CONTINUOUS) for i in F for t in T}
#total quantity from farm i stored in quarter t (so also the previous)
z = {(i,t) : m.add_var(var_type=mip.CONTINUOUS) for i in F for t in T0}

#OBJECTIVE FUNCTION
m.objective = mip.maximize(mip.xsum(( y[i,t] * kWhpt * ppkWh - x[i, t] * c[i] - z[i, t] * s[i]) for i in F for t in T))

#CONSTRAINTS

#for each quarter t and for each farm i the corn quantity bought is less than or equal than the quantity offered by the farm
for i in F:
  for t in T:
    m.add_constr(x[i, t] <= d[i][t])


#for each quarter t and for each farm i the corn quantity burned is greater or equal than 0
for i in F:
  for t in T:
    m.add_constr(x[i, t] >= 0)

#for each quarter t and for each farm i the corn quantity burned is greater or equal than 0
for i in F:
  for t in T:
    m.add_constr(y[i, t] >= 0)

#for each quarter t and for each farm i the corn quantity burned is greater or equal than 0
for i in F:
  for t in T:
    m.add_constr(z[i, t] >= 0)


#stored chopping equal to 0 at the beginning
for i in F:
  m.add_constr(z[i, -1] == 0)
#for each quarter stored quantity less than or equal max silos capacity
for t in T:
  m.add_constr(mip.xsum(z[i, t] for i in F) <= maxsc)

#for each quarter energy produced less than or equal max energy production
for t in T:
  m.add_constr(mip.xsum( y[i, t] * kWhpt for i in F) <= maxeprod)

#for each quarter dry matter percentage greater than or equal dry matter percentage down
for t in T:
  m.add_constr(mip.xsum( y[i, t] * dm[i] for i in F) >= mip.xsum((y[i,t]) * dmpd for i in F))
#for each quarter dry matter percentage less than or equal dry matter percentage up
for t in T:
  m.add_constr(mip.xsum( y[i, t] * dm[i] for i in F) <= mip.xsum( y[i,t] * dmpu for i in F))

#storage function
for t in T:
  for i in F:
    m.add_constr(z[i, t] == z[i, t-1] + x[i, t] - y[i, t])

m.optimize()

for i in F:
  print("        ", end = ' ')
  for t in T:
    print("%8.2f" % x[i, t].x, end = ' ')
  print("\n")
print("\n\n")
for i in F:
  for t in T0:
    print("%8.2f" % z[i, t].x, end = ' ')
  print("\n")
print("\n\n")
print(m.objective_value)

           700.00  1500.00   700.00     0.00     0.00   700.00  1500.00     0.00 

          1350.00     0.00   450.00     0.00  1350.00     0.00   450.00     0.00 

             0.00  1500.00  1500.00     0.00     0.00  1500.00  1500.00     0.00 

           820.00  1560.00   820.00     0.00   820.00  1560.00   820.00     0.00 

             0.00   565.24  1080.00     0.00     0.00   680.00   855.24     0.00 




    0.00     0.00   500.00   318.57    -0.00     0.00     0.00   375.00     0.00 

    0.00    -0.00     0.00    -0.00     0.00    -0.00     0.00    -0.00     0.00 

    0.00     0.00     0.00   106.19    -0.00     0.00     0.00   125.00     0.00 

    0.00     0.00     0.00    -0.00     0.00     0.00     0.00    -0.00     0.00 

    0.00     0.00     0.00    -0.00    -0.00     0.00     0.00    -0.00     0.00 




2861373.9254127136


In [4]:
## CODICE GRECYA
import mip
m = mip.Model()
#SETS
T =   [i for i in range(8)]
T0 =  [i for i in range(-1, 8)]
F =   [i for i in range(5)]

#PARAMETERS
d = [ [   700,    1500,   700,    0,      0,    700,    1500,   0],
      [   1350,   0,      450,    0,      1350, 0,      450,    0],
      [   0,      1500,   1500,   0,      0,    1500,   1500,   0],
      [   820,    1560,   820,    0,      820,  1560,   820,    0],
      [   0,      680,    1080,   0,      0,    680,    1080,   0]]

c =   [   0.2,    0.18,   0.19,   0.21,   0.23  ]
s =   [   0.002,  0.012,  0.007,  0.011,  0.015 ]
dm =  [   15,     28,     35,     37,     42    ]

dmpd = 20
dmpu = 40
kWhpt = 421.6
ppkWh = 0.28
maxeprod = 1950000
maxsc = 500

#VARIABLES
x = {(i,t) : m.add_var(var_type=mip.CONTINUOUS) for i in F for t in T}
r = {(i,t) : m.add_var(var_type=mip.CONTINUOUS) for i in F for t in T0}

#OBJECTIVE FUNCTION
# of = [((r[i][t-1] + x[i][t] - r[i][t]) * kWhpt * ppkWh - x[i][t] * c[i] - r[i][t] * s[i]) for i in F for t in T]
# m.objective = mip.maximize((mip.xsum((r[i, t-1] + x[i, t] - r[i, t]) * kWhpt * ppkWh) - mip.xsum(x[i, t] * c[i]) - mip.xsum(r[i, t] * s[i]) for i in F for t in T))
m.objective = mip.maximize(mip.xsum(((r[i, t-1] + x[i, t] - r[i, t]) * kWhpt * ppkWh - x[i, t] * c[i] - r[i, t] * s[i]) for i in F for t in T))

#CONSTAINTS

#for each quarter t and for each farm i the corn quantity bought is less than or equal than the quantity offered by the farm
for i in F:
  for t in T:
    m.add_constr(x[i, t] <= d[i][t])

#for each quarter t and for each farm i the corn quantity stored is less than or equal than the quantity bought plus the quantity stored previouly
for i in F:
  for t in T:
    m.add_constr(r[i, t] <= x[i, t] + r[i, t-1])

#stored chopping equal to 0 at the beginning (t=-1)
for i in F:
  m.add_constr(r[i, -1] == 0)
#for each quarter stored quantity less than or equal max silos capacity
for t in T:
  m.add_constr(mip.xsum(r[i, t] for i in F) <= maxsc)

#for each quarter energy produced less than or equal max energy production
for t in T:
  m.add_constr(mip.xsum((r[i, t-1] + x[i, t] - r[i, t]) * kWhpt for i in F) <= maxeprod)

#for each quarter dry matter percentage greater than or equal dry matter percentage down
for t in T:
  m.add_constr(mip.xsum((r[i, t-1] + x[i, t] - r[i, t]) * dm[i] for i in F) >= mip.xsum((r[i, t-1] + x[i, t] - r[i, t]) * dmpd for i in F))
#for each quarter dry matter percentage less than or equal dry matter percentage up
for t in T:
  m.add_constr(mip.xsum((r[i, t-1] + x[i, t] - r[i, t]) * dm[i] for i in F) <= mip.xsum((r[i, t-1] + x[i, t] - r[i, t]) * dmpu for i in F))

m.optimize()

for i in F:
  print("        ", end = ' ')
  for t in T:
    print("%8.2f" % x[i, t].x, end = ' ')
  print("\n")
print("\n\n")
for i in F:
  for t in T0:
    print("%8.2f" % r[i, t].x, end = ' ')
  print("\n")
print("\n\n")
print(mip.xsum(((r[i, t-1].x + x[i, t].x - r[i, t].x) * kWhpt * ppkWh - x[i, t].x * c[i] - r[i, t].x * s[i]) for i in F for t in T))
print(m.objective_value)


           700.00  1500.00   700.00     0.00     0.00   700.00  1500.00     0.00 

          1350.00     0.00   450.00     0.00  1350.00     0.00   450.00     0.00 

             0.00  1500.00  1500.00     0.00     0.00  1500.00  1500.00     0.00 

           820.00  1560.00   820.00     0.00   820.00  1560.00   820.00     0.00 

             0.00   565.24  1080.00     0.00     0.00   680.00   855.24     0.00 




    0.00     0.00   500.00   318.57     0.00     0.00     0.00   375.00     0.00 

    0.00     0.00     0.00     0.00     0.00     0.00     0.00     0.00     0.00 

    0.00     0.00     0.00   106.19     0.00     0.00     0.00   125.00     0.00 

    0.00     0.00     0.00     0.00     0.00     0.00     0.00     0.00     0.00 

    0.00     0.00     0.00     0.00     0.00     0.00     0.00     0.00     0.00 




+ 2861373.925412714
2861373.9254127145


In [3]:
import mip

# Luca Brembilla (10915890), Christian Confalonieri (10669451), Grecya D'Angiò (10651939)

# Parameters

# Capacity of each farm per quarter
c = [[700.0,1500.0,700.0,0.0,0.0,700.0,1500.0,0.0],
     [1350.0,0.0,450.0,0.0,1350.0,0.0,450.0,0.0],
     [0.0,1500.0,1500.0,0.0,0.0,1500.0,1500.0,0.0],
     [820.0,1560.0,820.0,0.0,820.0,1560.0,820.0,0.0],
     [0.0,680.0,1080.0,0.0,0.0,680.0,1080.0,0.0]] #ton
# Unit purchase price
p = [0.20,0.18,0.19,0.21,0.23] #$/ton
# Unit storage cost
s = [0.002,0.012,0.007,0.011,0.015] #$/ton
# Unit percentage of dry matter
d = [15.0,28.0,35.0,37.0,42.0] #%/ton
dry_matter_range = [20.0,40.0] #%
yeld = 421.6 #kWh
market_price = 0.28 #$/ton
state_regulations = 1950000.0 #kWh
total_silo_capacity = 500.0 #ton

def solve_productmix(c,p,s,d,dry_matter_range,yeld,market_price,state_regulations,total_silo_capacity):
  # Farms
  F = range(len(c))
  # Quarters
  Q = range(len(c[0]))

  m = mip.Model()

  # The default value of the variable lower bound (lb) is 0.0, so, 
  # it is not necessary to enter variable constraints greater than or equal to 0.0.

  # Number of tons purchased
  x = {(i,j) : m.add_var(name="x{}{}".format(str(i),str(j)),ub=c[i][j]) for i in F for j in Q}
  # Number of tons stored at the end of the quarter
  y = {(i,j) : m.add_var(name="y{}{}".format(str(i),str(j))) for i in F for j in range(len(c[0])+1)}
  # Number of tons burned = x[i,j]+y[i,j]-y[i,j+1]

  # Constraints 
  for i in F:
    m += y[i,0]==0.0
    for j in Q:
      # Burned material must be greater than or equal to 0
      m += x[i,j]+y[i,j]-y[i,j+1]>=0.0

  for j in Q:
    # The dry matter percentage must be between 20 and 40
    m += mip.xsum(d[i]*(x[i,j]+y[i,j]-y[i,j+1]) for i in F) >= dry_matter_range[0] * mip.xsum((x[i,j]+y[i,j]-y[i,j+1]) for i in F)
    m += mip.xsum(d[i]*(x[i,j]+y[i,j]-y[i,j+1]) for i in F) <= dry_matter_range[1] * mip.xsum((x[i,j]+y[i,j]-y[i,j+1]) for i in F)
    # Stocks per quarter shall not exceed 500 tons
    m += mip.xsum(y[i,j+1] for i in F) <= total_silo_capacity
    # State regulations must be met in each quarter: no more energy than 1950 MWh must be produced
    m += yeld * mip.xsum((x[i,j]+y[i,j]-y[i,j+1]) for i in F) <= state_regulations

  # Objective Function
  m.objective = mip.maximize(yeld*market_price*mip.xsum((x[i,j]+y[i,j]-y[i,j+1]) for i in F for j in Q)-mip.xsum(p[i]*x[i,j]+s[i]*y[i,j+1] for i in F for j in Q))
  m.optimize()

  return (m, [["%.2f" % x[i,j].x for j in Q] for i in F], [["%.2f" % y[i,j+1].x for j in Q] for i in F], m.objective_value)

_, solution_x, solution_y, objective = solve_productmix(c,p,s,d,dry_matter_range,yeld,market_price,state_regulations,total_silo_capacity)

print("Tons of material I decide to purchase (ton/quarter) (variable x):")
for line in solution_x:
  print("\t".join(map(str,line)))
print("\n")
print("Tons of material in the silo (ton/quarter) (variable y):")
for line in solution_y:
  print("\t".join(map(str,line)))
print("\n")  
print("Total profit ($) (objective value): ", objective)

Tons of material I decide to purchase (ton/quarter) (variable x):
700.00	1500.00	700.00	0.00	0.00	700.00	1500.00	0.00
1350.00	0.00	450.00	0.00	1350.00	0.00	450.00	0.00
0.00	1500.00	1500.00	0.00	0.00	1500.00	1500.00	0.00
820.00	1560.00	820.00	0.00	820.00	1560.00	820.00	0.00
0.00	565.24	1080.00	0.00	0.00	680.00	855.24	0.00


Tons of material in the silo (ton/quarter) (variable y):
0.00	500.00	318.57	0.00	0.00	0.00	375.00	0.00
0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
0.00	0.00	106.19	0.00	0.00	0.00	125.00	0.00
0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00


Total profit ($) (objective value):  2861373.9254127145
